In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

## Connect to EXACT

In [4]:
configuration = Configuration()
configuration.username = 'exact' #'exact'
configuration.password = 'exact' #'exact'
configuration.host = "http://127.0.0.1:1337" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

## Download Rect annotation

In [5]:
annotation_types_points = {}
annotation_types_ref_rect = None

for product in product_api.list_products(name="References").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id, name="BoundingBox").results:
        annotation_types_ref_rect = annotation_type
            
for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.name] = annotation_type

annotation_types_points

{'L0': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 21,
  'name': 'L0',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 'L1': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 22,
  'name': 'L1',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 'L2': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 23,
  'name': 'L2',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 'L3': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': Fal

In [6]:
annotation_types_ref_rect

{'area_hit_test': True,
 'closed': True,
 'color_code': '#FF0000',
 'default_height': 5000,
 'default_width': 5000,
 'enable_blurred': False,
 'enable_concealed': False,
 'id': 10,
 'name': 'BoundingBox',
 'node_count': 0,
 'product': 6,
 'sort_order': 0,
 'vector_type': 1}

In [7]:
annotations = []

image_set = image_sets_api.list_image_sets(name="Single_IHC", expand="product_set,product_set.annotationtype_set").results[0]


for image_id in image_set.images:
    
    image = images_api.retrieve_image(id=image_id)
    
    for anno in annotations_api.list_annotations(image=image_id, annotation_type=annotation_types_ref_rect.id, # BoundBox ID
                                                 deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:
        
        annotations.append([image.id, image.name, anno.id, anno.vector, anno.unique_identifier, anno.annotation_type])
        
annotations = pd.DataFrame(annotations, columns=["image_id", "image_name", "id", "vector", "unique_identifier", "annotation_type"])
annotations.head()

,image_id,image_name,id,vector,unique_identifier,annotation_type
0,324,CRC-AI-15 X40 IHC.tif,25977,"{'x1': 91, 'x2': 72457, 'y1': 2624, 'y2': 76976}",4d344740-52a8-4ef1-b194-8d1cf97e2766,10
1,326,CRC-AI-8 E X40 IHC.tif,25978,"{'x1': 4835, 'x2': 62584, 'y1': 1822, 'y2': 58...",20ddc2d1-fa7c-415a-a0ec-d42fb138ecf5,10
2,327,CRC-AI-8 H X40 IHC.tif,25979,"{'x1': 29, 'x2': 62223, 'y1': 2888, 'y2': 87710}",042796b5-5adf-4414-9e2c-7d8d4dd10e73,10
3,329,CRC-AI-11 F X40 IHC.tif,25976,"{'x1': 103, 'x2': 51830, 'y1': 2211, 'y2': 103...",7018a17f-6bf7-4816-90e6-a763e2b852d9,10
4,331,CRC-AI-14 1G X40 IHC.tif,25980,"{'x1': 76, 'x2': 65264, 'y1': 729, 'y2': 88611}",34042d48-9390-46c9-8ed5-944ea7af32c6,10


In [8]:
num_lines = 5
patch_size = 100

registration_points = []
for id, bounding_box in annotations.iterrows():
    
    box_width = bounding_box.vector["x2"] - bounding_box.vector["x1"]
    box_height = bounding_box.vector["y2"] - bounding_box.vector["y1"]
    
    step_size_x = box_width // num_lines
    step_size_y = box_height // num_lines
        
    point_index = 0
    for x in range(bounding_box.vector["x1"] + step_size_x //2, bounding_box.vector["x2"], step_size_x):
        for y in range(bounding_box.vector["y1"] + step_size_y //2, bounding_box.vector["y2"], step_size_y):
            
            ref_point = annotation_types_points[f"L{point_index}"]
                
            vector = {"x1": x - patch_size, "y1": y - patch_size, 
                          "x2": x + patch_size, "y2": y + patch_size}
                
            point = [bounding_box.image_id, vector, point_index, ref_point.name, ref_point.id]
                
            registration_points.append(point)
                
            point_index += 1
    
registration_points = pd.DataFrame(registration_points, 
                                   columns=["image_id", "vector", "point_index", "point_name", "point_id"])
registration_points

,image_id,vector,point_index,point_name,point_id
0,324,"{'x1': 7227, 'y1': 9959, 'x2': 7427, 'y2': 10159}",0,L0,21
1,324,"{'x1': 7227, 'y1': 24829, 'x2': 7427, 'y2': 25...",1,L1,22
2,324,"{'x1': 7227, 'y1': 39699, 'x2': 7427, 'y2': 39...",2,L2,23
3,324,"{'x1': 7227, 'y1': 54569, 'x2': 7427, 'y2': 54...",3,L3,24
4,324,"{'x1': 7227, 'y1': 69439, 'x2': 7427, 'y2': 69...",4,L4,25
...,...,...,...,...,...
120,331,"{'x1': 58642, 'y1': 9417, 'x2': 58842, 'y2': 9...",20,L20,41
121,331,"{'x1': 58642, 'y1': 26993, 'x2': 58842, 'y2': ...",21,L21,42
122,331,"{'x1': 58642, 'y1': 44569, 'x2': 58842, 'y2': ...",22,L22,43
123,331,"{'x1': 58642, 'y1': 62145, 'x2': 58842, 'y2': ...",23,L23,44


## Upload ref Points to EXACT

In [9]:
annotations_to_upload = []

for id, bounding_box in tqdm(registration_points.iterrows()):
    
    
    exact_annos =  annotations_api.list_annotations(image=bounding_box.image_id, annotation_type=bounding_box.point_id).results
    
    if len(exact_annos) == 0:
        anno = Annotation(annotation_type=bounding_box.point_id, 
                          vector=bounding_box.vector, 
                          image=bounding_box.image_id)

        annotations_to_upload.append(anno)

    else:
        prinf(f"ImageId: {bounding_box.image_id}, AnnotationType: {bounding_box.point_id} combination allready exists!")

125it [00:42,  2.98it/s]


In [10]:
annotations_to_upload[:3]

[{'annotation_type': 21,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 324,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': None,
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 7227, 'x2': 7427, 'y1': 9959, 'y2': 10159},
  'verified_by_user': None},
 {'annotation_type': 22,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 324,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': None,
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 7227, 'x2': 7427, 'y1': 24829, 'y2': 25029},
  'verified_by_user': None},
 {'annotation_type': 23,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'im

## Check that no error occurred and afterwards upload annotations

In [11]:
#annotations_api.create_annotation(body=annotations_to_upload)

[{'annotation_type': 21,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 25981,
  'image': 324,
  'last_edit_time': datetime.datetime(2021, 2, 9, 13, 27, 49, 982126),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 2, 9, 13, 27, 49, 982099),
  'unique_identifier': 'f52b1e31-3df8-4041-9902-66bc6edef761',
  'uploaded_media_files': [],
  'user': 2,
  'vector': {'x1': 7227, 'x2': 7427, 'y1': 9959, 'y2': 10159},
  'verified_by_user': 'False'},
 {'annotation_type': 22,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 25982,
  'image': 324,
  'last_edit_time': datetime.datetime(2021, 2, 9, 13, 27, 49, 982194),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 2, 9, 13, 27, 49, 982180),
  'unique_identifier': 'b8ca1cea-38ef-4818-b050-d77ef0e44e21',
  'uploaded_media_files': [],
  'user': 2,